#### Test notebook that generates .tif files and stores in /results folder. Check env TEST_TILE_PATH to .jp2 tile. Use **python37** kernel.

In [2]:
import os
import sys
import rasterio
import pyproj
import zipfile
import tempfile
import random

from rasterio.windows import Window
from rasterio import Affine
from shapely.geometry import Polygon
from shapely.ops import transform
from pathlib import Path

In [3]:
sys.version

'3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 02:25:08) \n[GCC 7.5.0]'

In [4]:
NB_USER = os.getenv('NB_USER')
print(NB_USER)

jovyan


In [11]:
TEST_TILE_PATH = os.getenv('TEST_TILE_PATH')
TEST_TILE_PATH

'/home/jovyan/work/test_tile/L1C_T36UYA_A017393_20200705T083602_TCI.jp2'

In [5]:
tetleha = [(
              36.63013458251953,
              49.87273390600898
            ),
            (
              36.81964874267578,
              49.87273390600898
            ),
            (
              36.81964874267578,
              49.98103318054302
            ),
            (
              36.63013458251953,
              49.98103318054302
            ),
            (
              36.63013458251953,
              49.87273390600898
            ), ]

forest = [(
              36.747894287109375,
              50.008400954338185
            ),
            (
              36.88556671142578,
              49.874282751903394
            ),
            (
              37.00366973876953,
              49.92647191927194
            ),
            (
              36.84711456298828,
              50.03707625174409
            ),
            (
              36.747894287109375,
              50.008400954338185
            ),
          ]
kharkiv = [(
              36.066741943359375,
              50.03773778732018
            ),
            (
              36.25556945800781,
              49.87295517274949
            ),
            (
              36.46568298339844,
              49.96447253142292
            ),
            (
              36.31874084472656,
              50.072565780723615
            ),
            (
              36.066741943359375,
              50.03773778732018
            ),
          ]

In [6]:
def crop(input_path, output_path, polygon):
    with rasterio.open(input_path) as src:
        x_min, y_min, x_max, y_max = polygon.bounds
        
        affine = Affine(src.transform[0],
                        src.transform[1],
                        x_min,
                        src.transform[3],
                        src.transform[4],
                        y_max)

        row_min, col_max = rasterio.transform.rowcol(src.transform, x_max, y_max)
        row_max, col_min = rasterio.transform.rowcol(src.transform, x_min, y_min, op=round, precision=6)
        row_min = row_min + 1
        row_max = row_max + 1
        
        write_window = Window.from_slices([row_min, row_max], [col_min, col_max])
        raster = src.read(window=write_window)
        kwargs = src.meta.copy()
        kwargs.update({
            'height': write_window.height,
            'width': write_window.width,
            "transform": affine,
            'driver': 'GTiff'
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(raster)

In [7]:
def convert(input_path, output_path, format='GTiff'):
    with rasterio.open(input_path) as src:
        raster = src.read()
        crs = str(src.crs)
                
        kwargs = src.meta.copy()
        kwargs.update({'driver': format})
        
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(raster)
        return crs

In [8]:
def to_crs(poly, target, current='EPSG:4326'):
    project = pyproj.Transformer.from_crs(pyproj.CRS(current), pyproj.CRS(target), always_xy=True).transform
    transformed_poly = transform(project, poly)
    return transformed_poly 

In [9]:
def generate(dir_, tile_path, aoi, location_name):
    
    stem = Path(tile_path).stem
    tif_path = f"{dir_}/{stem}.tif"    
    target_crs = convert(tile_path, tif_path)
    aoi = to_crs(Polygon(aoi), target_crs)

    output_path = f"/home/{NB_USER}/work/results/{stem}_{location_name}.tif"
    crop(tif_path, output_path, aoi)
    print(f"Gnerated file: {output_path}")

In [14]:
dir_ = f"/home/{NB_USER}/work/notebooks"

In [13]:
with tempfile.TemporaryDirectory(dir=dir_) as tmpdirname:
    print('Сreated temporary directory for calculations:', tmpdirname)
        
    locations = {'tetleha': tetleha, 'forest': forest, 'kharkiv': kharkiv}
    name = random.choice(list(locations.keys()))
    aoi = locations[name]
    
    generate(tmpdirname, TEST_TILE_PATH, aoi, name)

Сreated temporary directory for calculations: /home/jovyan/work/notebooks/tmpmwa7tii9
Gnerated file: /home/jovyan/work/results/L1C_T36UYA_A017393_20200705T083602_TCI_kharkiv.tif
